#### Emo Model Fine-tuning
(helper_models) 

In [1]:
import pandas as pd

train_data = pd.read_csv('train_emo_prompt.tsv', sep='\t')
eval_data = pd.read_csv('valid_emo_prompt.tsv', sep='\t')

> Plutchik's emotion:
- Anticipating: anticipating, anxious
- Joy: joyful, content
- Trust: trusting
- Fear: afraid, terrified
- Surprise: surprised
- Sad: sad, lonely, devastated
- Disgust: disgusted
- Anger: angry, annoyed, furious

In [2]:
plutchik = ['anticipating', 'joy', 'trust', 'fear', 'surprise', 'sad', 'disgust', 'anger']

def filter_data(df, label):
    # Define a dictionary to map the values
    mapping = {
        'anxious': 'anticipating',
        'joyful': 'joy',
        'content': 'joy',
        'trusting': 'trust',
        'afraid': 'fear',
        'terrified': 'fear',
        'surprised': 'surprise',
        'lonely': 'sad',
        'devastated': 'sad',
        'disgusted': 'disgust',
        'angry': 'anger',
        'annoyed': 'anger',
        'furious': 'anger'
    }
    # Replace values in the 'context' column using the dictionary
    df['context']  = df['context'].replace(mapping)
    filtered_df = df[df['context'].isin(label)]

    header = {'context': 'label', 'prompt': 'text'}
    filtered_df = filtered_df.rename(columns=header)
    return filtered_df

train_df = filter_data(train_data, plutchik)
eval_df = filter_data(eval_data, plutchik)
# train_data['context'] = train_data['context'].replace(mapping)
# eval_data['context'] = eval_data['context'].replace(mapping)

In [3]:
# len(df.context.unique())

In [4]:

# filtered_df = train_data[train_data['context'].isin(plutchik)]

# rename dataframe header
# header = {'context': 'label', 'prompt': 'text'}
# filtered_df = filtered_df.rename(columns=header)
# print(filtered_df)

In [5]:
# for i in plutchik:
#     cp = filtered_df[filtered_df['label'] == i]
#     print(f"{i}: {len(cp)}")

In [6]:
# Create id2label and label2id dictionaries
id2label = {i: emotion for i, emotion in enumerate(plutchik)}
label2id = {emotion: i for i, emotion in enumerate(plutchik)}

train_text = train_df.text.values
train_label = [label2id[label] for label in train_df.label.values]

eval_text = eval_df.text.values
eval_label = [label2id[label] for label in eval_df.label.values]

In [7]:
def encodings(texts, tokenizer):
    encoded_data = tokenizer(
        [text for text in texts],
        padding=True,
        truncation=True,
        return_tensors='pt',
        max_length=64
    )
    return encoded_data

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
train_encoding = encodings(train_text, tokenizer)
eval_encoding = encodings(eval_text, tokenizer)

In [9]:
from torch.utils.data import Dataset, DataLoader
    
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'labels': self.labels[idx]
        }

train_dataset = CustomDataset(train_encoding, train_label)
eval_dataset = CustomDataset(eval_encoding, eval_label)

In [10]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=len(label2id), id2label=id2label, label2id=label2id)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.we

In [11]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [12]:
import numpy as np
import evaluate

accuracy = evaluate.load("accuracy")
recall = evaluate.load('recall')
precision = evaluate.load("precision")
f1 = evaluate.load("f1")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    acc_score = accuracy.compute(predictions=predictions, references=labels)
    r_score = recall.compute(predictions=predictions, references=labels, average='macro')
    p_score = precision.compute(predictions=predictions, references=labels, average='macro')
    f1_score = f1.compute(predictions=predictions, references=labels, average='macro')

    metrics = {
        'accuracy':round(acc_score['accuracy'], 2),
        'precision':round(p_score['precision'], 2),
        'recall':round(r_score['recall'], 2),
        'f1':round(f1_score['f1'], 2)
    }
    return metrics

In [34]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback, IntervalStrategy

batch_size = 64
training_args = TrainingArguments(
    output_dir="results-30",
    learning_rate=2e-5,
    seed= 42,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy=IntervalStrategy.STEPS,
    eval_steps = 50,
    report_to="tensorboard",
    push_to_hub=False,
    save_total_limit=2,
    logging_dir='logs-30',
    metric_for_best_model = 'f1',
    load_best_model_at_end=True,
)
early_stop = EarlyStoppingCallback(2, 1.0)
# early_stop = EarlyStoppingCallback(early_stopping_patience=3)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[early_stop]
)

trainer.train()
trainer.save_model(output_dir='results-30/best-model')

  2%|▏         | 56/2740 [45:08<36:03:34, 48.37s/it]
/home/mauliana/anaconda3/envs/py2/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  4%|▎         | 51/1370 [00:12<17:24,  1.26it/s]

{'eval_loss': 2.0242905616760254, 'eval_accuracy': 0.77, 'eval_precision': 0.77, 'eval_recall': 0.76, 'eval_f1': 0.76, 'eval_runtime': 1.9151, 'eval_samples_per_second': 702.827, 'eval_steps_per_second': 11.488, 'epoch': 0.36}


  7%|▋         | 100/1370 [00:25<04:42,  4.50it/s]

{'eval_loss': 2.002079725265503, 'eval_accuracy': 0.77, 'eval_precision': 0.75, 'eval_recall': 0.77, 'eval_f1': 0.76, 'eval_runtime': 2.2571, 'eval_samples_per_second': 596.352, 'eval_steps_per_second': 9.747, 'epoch': 0.73}


 11%|█         | 150/1370 [00:37<04:38,  4.38it/s]

...

{'eval_loss': 1.9877229928970337, 'eval_accuracy': 0.77, 'eval_precision': 0.76, 'eval_recall': 0.76, 'eval_f1': 0.76, 'eval_runtime': 2.2273, 'eval_samples_per_second': 604.329, 'eval_steps_per_second': 9.878, 'epoch': 3.65}


 40%|████      | 550/1370 [02:41<03:04,  4.45it/s]

{'eval_loss': 1.9841604232788086, 'eval_accuracy': 0.76, 'eval_precision': 0.75, 'eval_recall': 0.76, 'eval_f1': 0.75, 'eval_runtime': 2.3168, 'eval_samples_per_second': 580.982, 'eval_steps_per_second': 9.496, 'epoch': 4.01}


 44%|████▍     | 600/1370 [02:54<03:43,  3.44it/s]


{'eval_loss': 1.9617804288864136, 'eval_accuracy': 0.77, 'eval_precision': 0.78, 'eval_recall': 0.76, 'eval_f1': 0.76, 'eval_runtime': 2.0584, 'eval_samples_per_second': 653.918, 'eval_steps_per_second': 10.688, 'epoch': 4.38}
{'train_runtime': 174.257, 'train_samples_per_second': 501.73, 'train_steps_per_second': 7.862, 'train_loss': 0.011466334263483683, 'epoch': 4.38}


In [39]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import logging

# Set the logging level to WARNING or higher to suppress INFO messages
logging.getLogger("transformers").setLevel(logging.WARNING)

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained('results-30/best-model')

new_text = "some guys shot my neighbour and ran into the woods"
inputs = tokenizer(new_text, padding=True, truncation=True, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

probabilities = torch.softmax(logits, dim=1)
predicted_class_index = torch.argmax(probabilities, dim=1).item()

class_labels = plutchik

predicted_class = class_labels[predicted_class_index]
predicted_probabilities = probabilities[0].tolist()

print("Predicted Class:", predicted_class)
print("Class Probabilities:")
# for label, prob in zip(class_labels, predicted_probabilities):
#     print(f"{label}: {prob:.4f}")
sorted_probabilities = sorted(
    zip(class_labels, predicted_probabilities),
    key=lambda x: x[1],
    reverse=True
)

for label, prob in sorted_probabilities:
    print(f"{label}: {prob:.4f}")


Predicted Class: fear
Class Probabilities:
fear: 1.0000
sad: 0.0000
surprise: 0.0000
disgust: 0.0000
anger: 0.0000
trust: 0.0000
anticipating: 0.0000
joy: 0.0000


In [1]:
from transformers import pipeline
import logging

logging.disable(logging.INFO)

text = "I am fine thank you"
classifier = pipeline("text-classification", model="results-10/best-model", return_all_scores=True)
predicted_label = classifier(text)

# Sort the list of dictionaries based on the 'score' in descending order
sorted_data = sorted(predicted_label[0], key=lambda x: x['score'], reverse=True)

# Print the sorted data with labels and scores rounded to 2 decimal places
print(f"Predicted label: {sorted_data[0]['label']}")
print('Label Probability:')
for item in sorted_data:
    label = item['label']
    score = round(item['score'], 4)
    print(f'{label}: {score}')


/home/mauliana/anaconda3/lib/python3.9/site-packages/transformers/pipelines/text_classification.py:89: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


Predicted label: joy
Label Probability:
joy: 1.0
trust: 0.0
surprise: 0.0
fear: 0.0
disgust: 0.0
anger: 0.0
anticipating: 0.0
sad: 0.0
